# Skin Disease Detection using Mobile Application
## Final Year Project 2
Ahmad Daniel Ikhwan Bin Rosli <br>
1201103071

In [10]:
import shutil
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, MobileNetV2, MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K


In [2]:
!kaggle datasets download -d shubhamgoel27/dermnet --unzip -p datasets/dermnet_raw

Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)


In [3]:
target_classes = {
    "Acne and Rosacea Photos": "Acne",
    "Eczema Photos": "Eczema",
    "Psoriasis pictures Lichen Planus and related diseases": "Psoriasis",
    "Tinea Ringworm Candidiasis and other Fungal Infections": "Tinea",
    "Melanoma Skin Cancer Nevi and Moles": "Melanoma"
}

raw_path = Path("datasets/dermnet_raw")     
prep_path = Path("datasets/dermnet_prep")    

In [13]:
if prep_path.exists():
    shutil.rmtree(prep_path)
prep_path.mkdir(parents=True)

for original_name, clean_name in target_classes.items():
    dst_dir = prep_path / clean_name
    dst_dir.mkdir(parents=True, exist_ok=True)
    
    img_counter = 0

    for split in ['train', 'test']:
        src_dir = raw_path / split / original_name
        if not src_dir.exists():
            continue

        for img_file in tqdm(list(src_dir.glob("*.jpg")), desc=f"Copying {clean_name} ({split})"):
            dst_file = dst_dir / f"{clean_name}_{img_counter:05d}.jpg"
            shutil.copy(img_file, dst_file)
            img_counter += 1

Copying Melanoma (test): 100%|██████████| 116/116 [00:01<00:00, 103.52it/s]


In [4]:
filepaths = []
labels = []

for class_folder in prep_path.iterdir():
    if class_folder.is_dir():
        for img_file in class_folder.glob("*.jpg"):
            filepaths.append(str(img_file))
            labels.append(class_folder.name)

df = pd.DataFrame({
    'filepath': filepaths,
    'label': labels
})

print(df.head())
print(df['label'].value_counts())


                                    filepath label
0  datasets\dermnet_prep\Acne\Acne_00000.jpg  Acne
1  datasets\dermnet_prep\Acne\Acne_00001.jpg  Acne
2  datasets\dermnet_prep\Acne\Acne_00002.jpg  Acne
3  datasets\dermnet_prep\Acne\Acne_00003.jpg  Acne
4  datasets\dermnet_prep\Acne\Acne_00004.jpg  Acne
label
Psoriasis    1757
Tinea        1625
Eczema       1544
Acne         1148
Melanoma      579
Name: count, dtype: int64


In [5]:
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print("Train:", train_df.shape)
print("Val:", val_df.shape)
print("Test:", test_df.shape)


Train: (5322, 2)
Val: (665, 2)
Test: (666, 2)


In [6]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.15,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
# create data generators 
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_gen = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5322 validated image filenames belonging to 5 classes.
Found 665 validated image filenames belonging to 5 classes.
Found 666 validated image filenames belonging to 5 classes.


In [12]:
# mobilenet
print("Training MobileNet (50 epochs)...")
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
num_classes = len(train_gen.class_indices)
preds = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(patience=20, restore_best_weights=True, verbose=1)

history_mobilenet = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stop]
)

model.save("mobilenet_model.h5")
K.clear_session()

Training MobileNet (50 epochs)...
Epoch 1/50


: 

In [ ]:
# mobilenetv2
print("Training MobileNetV2 (50 epochs)...")
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(patience=20, restore_best_weights=True, verbose=1)

history_mobilenetv2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stop]
)

model.save("mobilenetv2_model.h5")
K.clear_session()


In [ ]:
# mobilenetv3
print("Training MobileNetV3Small (50 epochs)...")
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(patience=20, restore_best_weights=True, verbose=1)

history_mobilenetv3 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stop]
)

model.save("mobilenetv3Large_model.h5")
K.clear_session()
